In [1]:
import sys
sys.path.append("../script/")

import Functions
from Engine import Engine
from GTGP import GTGP


from sklearn.metrics import classification_report,accuracy_score,confusion_matrix,f1_score
from sklearn.model_selection import train_test_split
import numpy as np
import importlib
from time import time
from matplotlib import pyplot as plt
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
import numpy as np
import pandas as pd
from time import time

In [2]:
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.utils import shuffle
def plot_roc_curve(true_y, y_prob):
    """
    plots the roc curve based of the probabilities
    """
    
    fpr, tpr, thresholds = roc_curve(true_y, y_prob)
    plt.plot(fpr, tpr)
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    print(roc_auc_score(true_y,prob[:,1]))

# data

In [3]:
df = pd.read_csv("../data/vowel.tsv",delimiter='\t')
X = df.iloc[:,:-1].to_numpy().astype("float")
y = df.iloc[:,-1].to_numpy().astype("int")

seeds = [10086, 200,500,30506,30405,30420,10056,7059,40965,5398,869543,83491,823190,
         48392,2810,48392,3498210,483902,859032,12890,538920,86954,54309,6504,9840,
         219805,548,2981,432890,5438908,219094,5843902,60854,979,12890,2108,4093]
train_size = 0.7
dataset = 'vowel'

In [4]:
pd.value_counts(y)

8     90
2     90
1     90
0     90
5     90
6     90
3     90
9     90
4     90
10    90
7     90
Name: count, dtype: int64

In [6]:
def to_one_hot(y,yt):
    le = LabelEncoder()
    ohe = OneHotEncoder()
    y_train = le.fit_transform(y)
    y_one_hot = ohe.fit_transform(y_train.reshape(-1,1))
    
    y_test = le.transform(yt)
    yt_one_hot = ohe.transform(y_test.reshape(-1,1))
    
    return y_train,y_test,y_one_hot,yt_one_hot

# model

In [23]:
def fit_trees():
    learning_rate=0.1
    max_depth=3
    bins=8
    lam=10

    gtgp = GTGP(learning_rate=learning_rate,max_depth=max_depth,bins=bins,lam=lam)

    total_size=10
    elite_size = 10
    epoch= 100
    gp_epoch= 3
    verbose = 1
    tolerance=0.01

    gtgp.fit(X_train,y_train,total_size=total_size,elite_size = elite_size,epoch=epoch,gp_epoch=gp_epoch,tolerance=tolerance,verbose=verbose)

    retrain_epoch= 5
    alpha=0
    beta=1
    gammer=0

    verbose=1
    gtgp.lam = 10
    gtgp.retrain_estimators(X_test,y_test,retrain_epoch=retrain_epoch,alpha=alpha,beta=beta,gammer=gammer,verbose=verbose)

    return gtgp

with open('./benchmark/'+dataset+'.csv','a') as f:
    for i in range(30):
        X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=train_size,stratify=y, random_state=seeds[i])
        gtgp = fit_trees()

        print(classification_report(y_train,np.argmax(gtgp.train_p,axis=1),digits=5))
        print(classification_report(y_test,np.argmax(gtgp.test_p,axis=1),digits=5))

        num_trees,depth,num_nodes = gtgp.print_model()

        train_acc = accuracy_score(y_train,np.argmax(gtgp.train_p,axis=1))
        test_acc = accuracy_score(y_test,np.argmax(gtgp.test_p,axis=1))
        train_f1 = f1_score(y_train,np.argmax(gtgp.train_p,axis=1),average='macro')
        test_f1 = f1_score(y_test,np.argmax(gtgp.test_p,axis=1),average='macro')

        s = str(train_acc)+","+str(test_acc)+","+str(train_f1)+","+str(test_f1)+","+str(num_trees)+","+str(depth)+","+str(num_nodes)+"\n"
        f.writelines(s)

27 125 1
53 247 2
82 390 3
107 513 4
133 629 5
158 740 6
181 843 7
205 973 8
224 1058 9
248 1174 10
275 1291 11
293 1387 12
311 1483 13
333 1591 14
349 1669 15
374 1810 16
394 1904 17
418 2024 18
438 2130 19
460 2262 20
476 2346 21
494 2460 22
513 2557 23
531 2653 24
548 2748 25
565 2835 26
578 2900 27
595 2977 28
614 3070 29
637 3207 30
658 3336 31
675 3429 32
695 3535 33
710 3620 34
729 3723 35
738 3770 36
755 3865 37
769 3939 38
782 4010 39
795 4083 40
810 4174 41
820 4234 42
833 4303 43
846 4382 44
865 4497 45
881 4585 46
896 4672 47
911 4755 48
928 4850 49
943 4943 50
959 5041 51
978 5152 52
989 5205 53
999 5253 54
1011 5323 55
1031 5443 56
1047 5537 57
1061 5625 58
1074 5706 59
1088 5788 60
1102 5876 61
1113 5941 62
1127 6029 63
1141 6113 64
1152 6182 65
1163 6255 66
1178 6340 67
1192 6424 68
1201 6475 69
1213 6541 70
1227 6617 71
1242 6706 72
1256 6788 73
1273 6905 74
1285 6981 75
1298 7070 76
1308 7130 77
1319 7201 78
1330 7272 79
1340 7344 80
1352 7426 81
1365 7519 82
1379 760

# Decision Tree

In [8]:
from sklearn.tree import DecisionTreeClassifier

In [9]:

with open('./benchmark_DC/'+dataset+'.csv','w') as f:
        
    for i in range(30):
        X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=train_size,stratify=y, random_state=seeds[i])
        clf = DecisionTreeClassifier()
        clf.fit(X_train,y_train)

        num_trees = 1
        depth = clf.tree_.max_depth
        num_nodes = clf.tree_.node_count

        train_acc = accuracy_score(y_train,clf.predict(X_train))
        test_acc = accuracy_score(y_test,clf.predict(X_test))
        train_f1 = f1_score(y_train,clf.predict(X_train),average='macro')
        test_f1 = f1_score(y_test,clf.predict(X_test),average='macro')

        s = str(train_acc)+","+str(test_acc)+","+str(train_f1)+","+str(test_f1)+","+str(num_trees)+","+str(depth)+","+str(num_nodes)+"\n"
        f.writelines(s)

# XGBoost

In [10]:
import xgboost

In [11]:
with open('./benchmark_xgb/'+dataset+'.csv','w') as f:
    for i in range(30):
        X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=train_size,stratify=y, random_state=seeds[i])
        xgb = xgboost.XGBClassifier(n_estimators=100,max_depth=6)
        # xgb = xgboost.XGBClassifier(min_child_weight=0,max_depth=4)
        # xgb = xgboost.XGBClassifier()
        xgb.fit(X_train,y_train)

        import json

        def item_generator(json_input, lookup_key):
            if isinstance(json_input, dict):
                for k, v in json_input.items():
                    if k == lookup_key:
                        yield v
                    else:
                        yield from item_generator(v, lookup_key)
            elif isinstance(json_input, list):
                for item in json_input:
                    yield from item_generator(item, lookup_key)

        def tree_depth(json_text):
            json_input = json.loads(json_text)
            depths = list(item_generator(json_input, 'depth'))
            return max(depths) + 1 if len(depths) != 0 else 1

        train_acc = accuracy_score(y_train,xgb.predict(X_train))
        test_acc = accuracy_score(y_test,xgb.predict(X_test))
        train_f1 = f1_score(y_train,xgb.predict(X_train),average='macro')
        test_f1 = f1_score(y_test,xgb.predict(X_test),average='macro')

        booster = xgb.get_booster()

        tree_df = booster.trees_to_dataframe()
        depths = [tree_depth(x) for x in booster.get_dump(dump_format = "json")]
        num_trees = len(depths)
        depth = np.average(depths)
        num_nodes = len(tree_df)

        s = str(train_acc)+","+str(test_acc)+","+str(train_f1)+","+str(test_f1)+","+str(num_trees)+","+str(depth)+","+str(num_nodes)+"\n"
        f.writelines(s)

# GDBT

In [24]:
from sklearn.ensemble import GradientBoostingClassifier

In [25]:
with open('./benchmark_GBDT/'+dataset+'.csv','w') as f:
    for i in range(30):
        X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=train_size,stratify=y, random_state=seeds[i])
        # clf = GradientBoostingClassifier(n_estimators=1000)
        clf = GradientBoostingClassifier()
        clf.fit(X_train,y_train)

        train_acc = accuracy_score(y_train,clf.predict(X_train))
        test_acc = accuracy_score(y_test,clf.predict(X_test))
        train_f1 = f1_score(y_train,clf.predict(X_train),average='macro')
        test_f1 = f1_score(y_test,clf.predict(X_test),average='macro')

        num_trees = len([ est for ests in clf.estimators_ for est in ests])
        depth = np.average([ max(1,est.tree_.max_depth) for ests in clf.estimators_ for est in ests])
        num_nodes = sum([ est.tree_.node_count for ests in clf.estimators_ for est in ests])

        s = str(train_acc)+","+str(test_acc)+","+str(train_f1)+","+str(test_f1)+","+str(num_trees)+","+str(depth)+","+str(num_nodes)+"\n"
        f.writelines(s)

# Random Forest

In [26]:
from sklearn.ensemble import RandomForestClassifier

In [27]:
with open('./benchmark_RF/'+dataset+'.csv','w') as f:
    for i in range(30):
        X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=train_size,stratify=y, random_state=seeds[i])
        rfc = RandomForestClassifier()
        rfc.fit(X_train,y_train)

        train_acc = accuracy_score(y_train,rfc.predict(X_train))
        test_acc = accuracy_score(y_test,rfc.predict(X_test))
        train_f1 = f1_score(y_train,rfc.predict(X_train),average='macro')
        test_f1 = f1_score(y_test,rfc.predict(X_test),average='macro')

        num_trees = len(rfc.estimators_)
        depth = np.average([est.tree_.max_depth for est in rfc.estimators_])
        num_nodes = sum([est.tree_.node_count for est in rfc.estimators_])

        s = str(train_acc)+","+str(test_acc)+","+str(train_f1)+","+str(test_f1)+","+str(num_trees)+","+str(depth)+","+str(num_nodes)+"\n"
        f.writelines(s)